### Run the Data Extraction and Pre-processing File

In [ ]:
%run ./x22163549_Ashlyn_fda_extraction_and_preprocessing.ipynb

### Visualization

#### 1) What are the most common reasons for recalls which provides insights into the types of issues that occur generally?

In [ ]:
text_col = 'reason_for_recall'

# Concatenate all the text from the selected column
text = ' '.join(fda_df_extracted[text_col].dropna().tolist())

# Generate word cloud
wordcloud = WordCloud(width=800, height=400, background_color='white').generate(text)

# Create a plot
fig, ax = plt.subplots(figsize=(10, 5))
ax.imshow(wordcloud, interpolation='bilinear')
ax.set_title('Word Cloud of ' + text_col.capitalize())
ax.axis('off')

# Display the plot
plt.show()

#### 2) What are the top 10 recalling firms with the highest number of recalls?

In [ ]:
# Get the top 10 recalling firms
top_10_recalling_firms = fda_df_extracted['recalling_firm'].value_counts().head(10)

# Create a bar chart
sns.barplot(x=top_10_recalling_firms.values, y=top_10_recalling_firms.index, palette='viridis')
plt.title('Top 10 Recalling Firms')
plt.xlabel('Recall Count')
plt.ylabel('Recalling Firm')
plt.show()

#### 3) What is the distribution of recalling firms across different initial firm notification categories? Which notification category has the highest number of firms, and which one has the lowest number of firms?

In [ ]:
# Group by initial firm notification and count the number of unique firms in each category
firms_per_notification = fda_df_extracted.groupby('initial_firm_notification')['recalling_firm'].nunique()

# Sort by number of firms in descending order
firms_per_notification = firms_per_notification.sort_values(ascending=False)

# Create a bar chart
plt.figure(figsize=(12, 6))
sns.barplot(x=firms_per_notification.index, y=firms_per_notification.values)
plt.xlabel('Initial Firm Notification')
plt.ylabel('Number of Unique Recalling Firms')
plt.title('Number of Firms per Initial Firm Notification Category (Ordered by Descending Count)')
plt.xticks(rotation=90)
plt.show()

#### 4) What are the trends or patterns of recalls over time i.e. How has the number of recalls changed over time? Are there any noticeable spikes or dips in recall activities?

In [ ]:
# Group the data by date and count the number of recalls in each date
recall_initiation_date_counts = fda_df_extracted['recall_initiation_date'].value_counts().sort_index()

# Create a time series plot for Recall Initiation Date
fig, ax = plt.subplots(figsize=(10, 5))
ax.plot(recall_initiation_date_counts.index, recall_initiation_date_counts.values, color="#660066")
ax.set_title('Recalls by Recall Initiation Date')
ax.set_xlabel('Recall Initiation Date')
ax.set_ylabel('Number of Recalls')

# Display the plots
plt.show()

#### 5) What is the distribution of product recalls by month in the past year? Months that experienced higher recall activity compared to others.

In [ ]:
# Calculate the past year date range
current_year = pd.Timestamp.now().year
past_year_start = pd.Timestamp(year=current_year-1, month=1, day=1)
past_year_end = pd.Timestamp(year=current_year-1, month=12, day=31)

# Filter the data for the past year
filtered_data = fda_df_extracted[
    (fda_df_extracted['recall_initiation_date'] >= past_year_start) &
    (fda_df_extracted['recall_initiation_date'] <= past_year_end)
]

# Group the filtered data by month and count the occurrences
monthly_recalls = filtered_data['recall_initiation_date'].dt.month.value_counts().sort_index()

# Create a 2D array for the heat map data
heat_map_data = np.zeros((12,))
for month, count in monthly_recalls.items():
    heat_map_data[month-1] = count

# Create the heat map
plt.figure(figsize=(10, 6))  # Set the figure size
plt.imshow([heat_map_data], cmap='magma', aspect='auto', interpolation='nearest')  # Plot the heat map
plt.colorbar(label='Recall Count')  # Add a colorbar with label
plt.title('Product Recalls by Month (Past Year)')  # Set the title of the chart
plt.xlabel('Month')  # Set the x-axis label
plt.xticks(np.arange(12), ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])  # Set x-axis tick labels
plt.ylabel('Recall Count')  # Set the y-axis label
plt.show()  # Show the plot

#### 6) What are the top 50 event IDs with the most recalls?

In [ ]:
# Group by event ID and count the unique recall numbers
event_recall_counts = fda_df_extracted.groupby('event_id')['recall_number_digits'].nunique().reset_index()

# Sort by recall counts in descending order and select the top 50 event IDs
top_50_event_ids = event_recall_counts.sort_values('recall_number_digits', ascending=False).head(50)

# Create a count plot
plt.figure(figsize=(12, 6))
sns.barplot(x='event_id', y='recall_number_digits', data=top_50_event_ids)
plt.xlabel('Event ID')
plt.ylabel('Number of Unique Recall Numbers')
plt.title('Top 50 Event IDs with Highest Number of Unique Recall Numbers')
plt.xticks(rotation=90)
plt.show()

#### 7) Which classification category (risk level indicator), has the highest or lowest percentage of recalls?

In [ ]:
# Count recalls by classification
df_counts = fda_df_extracted['classification'].value_counts()

custom_colors = ["#990033", "#009999", "#ff6600", "#33cc33", "#333399"]

# Pie chart
plt.figure(figsize=(8, 8)) # Set figure size
plt.pie(df_counts.values, labels=df_counts.index, autopct='%1.1f%%', startangle=90, colors=custom_colors) # Create pie chart
plt.title('Recalls Distribution by Classification') # Set plot title
plt.show() # Show plot

### Table Creation and Data Loading into PostgreSQL

In [ ]:
def create_fda_table(conn, table_name):
    try:
        cur = conn.cursor()
        cur.execute(f'''
            CREATE TABLE IF NOT EXISTS {table_name} (
                country VARCHAR(255),
                city VARCHAR(255),
                address_1 VARCHAR(255),
                reason_for_recall VARCHAR(255),
                product_quantity VARCHAR(255),
                code_info VARCHAR(255),
                center_classification_date DATE,
                distribution_pattern VARCHAR(255),
                state VARCHAR(255),
                product_description VARCHAR(255),
                report_date DATE,
                classification VARCHAR(255),
                recalling_firm VARCHAR(255),
                recall_number VARCHAR(255),
                initial_firm_notification VARCHAR(255),
                product_type VARCHAR(255),
                event_id VARCHAR(255),
                recall_initiation_date DATE,
                postal_code VARCHAR(255),
                voluntary_mandated VARCHAR(255),
                status VARCHAR(255),
                termination_date DATE,
                recall_number_digits INT
            )
        ''')
        conn.commit()
        print("Table created successfully!")
    except psycopg2.Error as e:
        print("Error creating table:", e)
        conn.rollback()
    finally:
        cur.close()
        
table_name = postgresFdatablename
create_fda_table(conn, table_name)

In [ ]:
def load_data_to_postgresql(conn, df, table_name):
    try:
        engine = establish_postgres_connection(postgresUsername, postgresPassword, postgresHost, postgresPort, postgresDbname)
        df.to_sql(table_name, engine, if_exists='replace', index=False)
        engine.dispose()
        print("Data loaded to PostgreSQL successfully!")
    except sqlalchemy.Error as e:
        print("Error loading data to PostgreSQL:", e)

load_data_to_postgresql(conn, fda_df_extracted, table_name)

In [ ]:
cur.close()
conn.close()